# 필요 library import

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# 입력일시, 제목, 기사내용, summary crawling 함수

- 기사 입력일시: 수정일시는 없는 기사도 많았으므로 입력일시를 사용
- 기사 자동요약이 없는 경우에는 기사는 데이터 수집하지 않도록 구현

In [74]:
def daum_crawl(url):
    # url 받아서 DOM 객체 만들기
    html = requests.get(url).text
    bs = BeautifulSoup(html, 'html.parser')
    
    # summary 유무 확인하고 있으면 data 가져오기
    if not bs.select('.btn_summary + div p'):
        return None
    else:
        # 입력일시, 제목
        date = bs.select_one('.info_view > span + span > span').text.strip()
        title = bs.select_one('em + h3').text.strip()  

        # 기사내용
        article_list = []
        for _ in bs.select('.article_view > section > p'):
            article_list.append(_.text.strip())
        article_list.pop() # 기자 이메일 제거
        article = '\n'.join(article_list)
        
        # summary
        summary_list = []
        for _ in bs.select('.btn_summary + div p'):
            summary_list.append(_.text.strip())
        summary = '\n'.join(summary_list)

        row = [date, title, article, summary]

    return row

def make_crawed_df(url_list):
    col_names = ['date', 'title', 'article', 'summary']
    rows = []
    
    for _ in url_list:
        if daum_crawl(_) != None:
            rows.append(daum_crawl(_))
        
    df = pd.DataFrame(rows, columns=col_names)
    
    return df

In [75]:
url_list = ['https://news.v.daum.net/v/20200728064119204', 'https://news.v.daum.net/v/20200728171755863',
           'https://news.v.daum.net/v/20200728194024562', 'https://news.v.daum.net/v/20200728214416635']
make_crawed_df(url_list)

,date,title,article,summary
0,2020.07.28. 06:41,천정부지 금값 온스당 2000달러 향해..달러 2년래 최저,(서울=뉴스1) 신기림 기자 = 금값이 뉴욕 거래에서 온스당 2000달러를 향해 돌...,금값이 뉴욕 거래에서 온스당 2000달러를 향해 돌진하며 사상 최고를 다시 갈아 치...
1,2020.07.28. 17:17,"파우치 ""백신개발 이르면 10월 가능..안전성 우려 안해""",미국 전염병 최고 권위자인 앤서니 파우치 국립알레르기·전염병연구소(NIAID) 소장...,미국 전염병 최고 권위자인 앤서니 파우치 국립알레르기·전염병연구소(NIAID) 소장...
2,2020.07.28. 19:40,"뉴질랜드 총리 ""WTO사무총장 선출 관심"" 문대통령 ""유명희 적임""",(서울=연합뉴스) 박경준 기자 = 문재인 대통령이 28일 오후 저신다 아던 뉴질랜드...,문재인 대통령이 28일 오후 저신다 아던 뉴질랜드 총리와 정상통화를 하고 세계무역기...


# 기사 링크 크롤링 -> url_list에 저장